<a href="https://colab.research.google.com/github/darshanp0411/Airline-Customer-Strategy-Intelligence-Agent/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# ==========================================
# 1. PAGE SETUP & SIDEBAR
# ==========================================
st.set_page_config(page_title="AAdvantage Strategy Dashboard", layout="wide")

st.title("✈️ Airline Customer Strategy & Intelligence Agent")
st.markdown("""
**Objective:** Strategic analysis of customer feedback to inform **segmentation**, **competitive intelligence**, and **roadmap prioritization**.
*Prototype built for the Airline Industry.*
""")

# SIDEBAR FOR API KEY (Best Practice)
with st.sidebar:
    st.header("⚙️ Settings")
    openai_api_key = st.text_input("OpenAI API Key (Optional)", type="password", help="API KEY")
    st.info("If no key is provided, the dashboard uses a Rule-Based demo mode.")

# ==========================================
# 2. LOAD DATA
# ==========================================
try:
    df = pd.read_csv("AAdvantage_Strategic_Analysis.csv", engine='python', on_bad_lines='warn')
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
except FileNotFoundError:
    st.error("⚠️ CSV file not found. Please run the analysis script first!")
    st.stop()

# ==========================================
# 3. EXECUTIVE SUMMARY (KPIs)
# ==========================================
st.divider()
col1, col2, col3, col4 = st.columns(4)

total_reviews = len(df)
biz_count = len(df[df['Traveler_Type'] == 'Business'])
biz_share = (biz_count / total_reviews) * 100 if total_reviews > 0 else 0
comp_count = len(df[df['Competitor'] != 'None'])

col1.metric("Total Reviews Analyzed", f"{total_reviews:,}")
col2.metric("Business Traveler Share", f"{biz_share:.1f}%")
col3.metric("Competitor Mentions", f"{comp_count}")
col4.metric("Avg Sentiment Score", f"{df['Sentiment_Score'].mean():.2f}")

st.divider()

# ==========================================
# 4. VISUALIZATIONS
# ==========================================
# ROW 1
c1, c2 = st.columns(2)
with c1:
    st.subheader("🔍 Segmentation Analysis")
    fig_seg = px.pie(df, names='Traveler_Type', title='Traveler Segment Distribution', color_discrete_sequence=px.colors.qualitative.Pastel)
    st.plotly_chart(fig_seg, use_container_width=True)

with c2:
    st.subheader("⚔️ Competitive Intelligence")
    comp_df = df[df['Competitor'] != 'None']
    if not comp_df.empty:
        fig_comp = px.bar(comp_df['Competitor'].value_counts().reset_index(), x='Competitor', y='count', title="Competitor Mentions", color='count', color_continuous_scale='Reds')
        st.plotly_chart(fig_comp, use_container_width=True)
    else:
        st.info("No competitor mentions detected.")

# ROW 2
c3, c4 = st.columns(2)
with c3:
    st.subheader("⚠️ Priority Pain Points")
    issue_counts = df['Key_Issue'].value_counts().reset_index()
    fig_issue = px.bar(issue_counts, x='count', y='Key_Issue', orientation='h', title="Top Complaint Categories", text='count')
    st.plotly_chart(fig_issue, use_container_width=True)

with c4:
    st.subheader("📈 Sentiment Trend")
    if 'Date' in df.columns:
        df['Month'] = df['Date'].dt.to_period('M').astype(str)
        trend_data = df.groupby('Month')['Sentiment_Score'].mean().reset_index()
        fig_trend = px.line(trend_data, x='Month', y='Sentiment_Score', markers=True, title="Average Sentiment Over Time")
        st.plotly_chart(fig_trend, use_container_width=True)
    else:
        st.warning("Date column missing.")

# ==========================================
# 5. LIVE AI AGENT (ChatGPT Integration)
# ==========================================
st.divider()
st.header("🤖 Ask the Strategic Analyst Agent")

# Show status
if openai_api_key:
    st.success("✅ **Live AI Mode Active:** Connected to GPT-3.5. You can ask complex questions.")
else:
    st.warning("⚠️ **Demo Mode:** Using keyword logic. Enter API Key in sidebar for full AI power.")

user_question = st.text_input("Type your question here (e.g., 'Draft an email to business travelers apologizing for delays'):", key="user_input")

if user_question:
    # --- OPTION A: REAL AI (If Key Exists) ---
    if openai_api_key:
        try:
            # 1. Initialize the Brain
            llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key=openai_api_key)

            # 2. Prepare the Data Context (We feed it stats + sample reviews)
            # We don't send the whole file (too big), we send the calculated stats + top 20 reviews
            context_stats = f"""
            Total Reviews: {total_reviews}
            Business Share: {biz_share:.1f}%
            Top Competitor: {df['Competitor'].value_counts().idxmax() if not df.empty else 'None'}
            Top Complaint: {df['Key_Issue'].value_counts().idxmax() if not df.empty else 'None'}
            """
            sample_reviews = df[['Traveler_Type', 'Competitor', 'Key_Issue', 'review']].head(20).to_string()

            # 3. Create the Prompt
            prompt_template = ChatPromptTemplate.from_template("""
            You are a Senior Airline Strategist for American Airlines.

            Here is the current data summary:
            {stats}

            Here is a sample of recent customer reviews:
            {reviews}

            User Question: {question}

            Answer the user's question based on this data. If asked to write content (emails, memos), be professional and empathetic.
            """
            )

            # 4. Get Answer
            response = llm.invoke(prompt_template.format(stats=context_stats, reviews=sample_reviews, question=user_question))

            # 5. Display
            st.markdown(response.content)

        except Exception as e:
            st.error(f"AI Error: {e}. Check your API Key.")

    # --- OPTION B: RULE-BASED FALLBACK (If No Key) ---
    else:
        q = user_question.lower()
        if "business" in q:
            top_issue = df[df['Traveler_Type']=='Business']['Key_Issue'].value_counts().idxmax()
            st.info(f"**Insight:** Business travelers are complaining about **{top_issue}**.")
        elif "competitor" in q:
             st.error("**Alert:** Customers are comparing us to Delta and United.")
        else:
            st.write("🤖 **Demo Bot:** I detected your question. (Enter API Key to get a full GPT response!)")

# ==========================================
# 6. RAW DATA
# ==========================================
with st.expander("📂 View Raw Strategic Data"):
    st.dataframe(df)

2025-12-16 21:35:43.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.973 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 21:35:43.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar